In [ ]:
import os
import astropy.units as u
import numpy as np

import pastis.util as util    
from pastis.config import CONFIG_PASTIS 
from pastis.e2e_simulators.luvoir_imaging import LuvoirA_APLC

import exoscene.star

In [ ]:
sampling = 4
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
luvoir = LuvoirA_APLC(optics_input, 'small', sampling)

In [ ]:
# Getting the flux together
sptype = 'A0V'
Vmag = 0.0
minlam = 500 * u.nanometer
maxlam = 600 * u.nanometer
star_flux = exoscene.star.bpgs_spectype_to_photonrate(spectype=sptype, Vmag=Vmag,
                                                      minlam=minlam.value, maxlam=maxlam.value)
npup = np.int(np.sqrt(luvoir.pupil_grid.x.shape[0]))
Nph = star_flux.value*15**2*np.sum(luvoir.apodizer**2) / npup**2

In [ ]:
star_flux.unit

In [ ]:
def open_loop_drift(kappa, flux_star, contrast_ratio, eta, sensitivity):
    drift = (1/kappa**3) * (flux_star * contrast_ratio**(3/2))/(2*eta**(3/2)*sensitivity)
    return drift

def closed_loop_drift(kappa, flux_star, contrast_ratio, eta, sensitivity, beta):
    drift = (1/kappa**2) * (flux_star * contrast_ratio**(3/2))/(2*eta**(1/2)*beta**2*sensitivity**3)
    return drift

In [ ]:
beta = 1
kappa = 10

c_0 = 1e-10
F_r = 4e-11
eta = c_0 / F_r

In [ ]:
# Trying a case
coro_lambda = 1e-6 * luvoir.wvln
OL_drift = open_loop_drift(kappa, Nph, F_r, eta, coro_lambda)
print(OL_drift)